# Modelado de los residuos restantes de los cristales


In [1]:
# Librerías
from modeller import * # licencia modeller necesaria
from prody import *
import csv
from modeller import * # licencia modeller necesaria
from modeller.automodel import *
from prody import * 
from Bio import pairwise2, SeqIO
import os
import glob

/home/joel/anaconda3/envs/mds/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Obtener la secuencia de UniProt
Utilizamos la secuencia P24941 correspondiente a la CDK2 humana.

In [2]:
# Secuencia de la CDK2 de UniProt
fasta_cdk2 = SeqIO.read(open('./B_DATOS/P24941.fasta'),'fasta')
cdk2_P24941 = str(fasta_cdk2.seq)
print(cdk2_P24941)

MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNHPNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHSHRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYYSTAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSFPKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL


### Función find gaps
Cargamos la función que permite permite identificar los gaps en una secuencia y devolver el rango del gap.

In [3]:
from Funciones.find_gaps import find_gaps
# Recibe una secuencia (con gaps como "-") y un valor r para valorar el gap en +/- r posiciones 

In [4]:
# EJEMPLO:
find_gaps("1234-5", r=2)

{'num_gaps': 1,
 'gap_lengths': [2],
 'gap_list': [[5, 6]],
 'gap_window': [[3, 6]]}

### Obtenemos los PDB IDs de los cristales
A partir de la tabal de metadatos generada con *Get_Metadatos..*.  
Tomar en cuenta que de los **408** cristales identificados, **seis han sido removidos**, debido a que el modelado no era posible:
- 5ang, 4ek6, 6ath, 5uq3, 1jsu, 5mhq.

In [7]:
# Los IDs de los cristales están en el archivo
import pandas as pd
df_cdk2 = pd.read_csv("B_DATOS/TABLA_MTDATA_CDK2_402_crys.csv", index_col=1)
pdbids_list = df_cdk2.index
print("Total de PDB IDs a modelar:", len(pdbids_list))

Total de PDB IDs a modelar: 402


## Se ejecuta Modeller para refinamiento por loops
#### Descripción de los pasos a ejecutar:
- **1)** Se definen los directorios de entradas y salidas.
- **2)** Se carga la estructura cristalográfica a modelar (*cristal*), correspondiente a cada pdb id.
    - Si la proteína ya fue modelada se omite volver a modelarla.
- **3)** Se hace un alineamiento de la secuencia del cristal y la secuecnia **P24941**.
- **4)** Se identifican los gaps en la secuencia del cristal, si los hay. Se define una ventana de $\pm$ *r* residuos que se considerarán dentro del gap.
- **5)** Se modifica el método *select_atoms* de la clase *MyModel* de MODELLER para indicar que únicamente modele los residuos que pertenecen al gap del cristal, dejando intacto el resto de los átomos de la proteína.
- **6)** Ejecuta MODELLER con los parámetros indicados, generando únicamente un modelo, el cual es guardado como _PDBID_**_MODLL.pdb**.

In [ ]:
import os
import time
start = time.time()

# Directiros
struct_dir = '../ARCHIVOS/CRISTALES/PROT_CDK2_CHAINS/'
tail_pdb = '_A.pdb' # extensión del archivo pdb del cristal
tail_model = '_MODLL' # epiteto del archivo del modelo final, la extensión .pdb la agrega modeller
pdbs_model_dir = "../ARCHIVOS/CRISTALES/PROT_CDK2_MODELOS_modeller_NOPREP/" # Carpeta de salida para los modelos
num_res_window = 2

for pdb_code in pdbids_list:
    ###########################
    if os.path.isfile(pdbs_model_dir + pdb_code + tail_model + ".pdb"):
        print("El modelo ya existe en el directorio")
        continue

    ###########################
    try:
        # Se lee el archivo pdb. La selección omite residuos negativos.
        # Además omite los residuos no estandar, con modificaciones postTrad como el TPO (Tirosina fosfatada)
        stc_prot = parsePDB(struct_dir 
                            + pdb_code + tail_pdb).select('not nonstdaa and resid > 0') 
        chaid = stc_prot.getChids()[0] # Obtiene el id de la cadena
        ref_hv = stc_prot.getHierView()[chaid] # Se obtiene sólo la cadena
        seq_cry = ref_hv.getSequence() # Se obtiene la secuencia de la estructura cristalográfica   
    except:
        print("Error al abrir y modelar:", pdb_code)
        continue
    ########################### 
    # Pregunta si la longitud y la secuencias de Uniprot y de la estructura son iguales
    same_seq = len(seq_cry) == len(cdk2_P24941) and seq_cry == cdk2_P24941
    # si same_seq es verdadero, se omite hacer el modelado pues la estructura está completa
    # y se pasa a la siguiente secuencia
    if same_seq:
        print("La proteína " + pdb_code + " ya está completa")
        continue  
        ''' SI HACEN FALTA ÁTOMOS A UN RESIDUO, SE AGREGARÁN CON pdb4amber'''
    print("Modelando proteína " + pdb_code)
    ###########################
    # Obtener la secuecnia de la estructura a modelar y guardarla en un archivo
    # Alineamiento global, se penalizan con -10 los gaps abiertos, se obtiene el mejor
    alignment = pairwise2.align.globalxs(seq_cry, cdk2_P24941, -10, -0.5, gap_char='-')[0]
    # Secuencias del alineamiento
    algn1_struc = alignment[0] # Secuencia alineada de la estructura cristalográfica
    algn2_seq = cdk2_P24941 # Secuencia completa de UniProt

    # Nombre de los cabezales de las secuencia deben coincidir con el de los archivos de entrada y salida
    crys_file_name = pdb_code + tail_pdb
    model_file_name = pdb_code + tail_model

    ''' NECESARIO: There should be 10 fields separated by colons ":".
    Please check the file to make sure your sequences end with the '*' character.
    Nomenclaturas de los campos del header: https://salilab.org/modeller/8v2/manual/node176.html'''

    # HEADERS (la secuencia del cristal va primero, luego la secuencia completa)
    struc_header = "structureX:" + crys_file_name + ":.:" + chaid + ":.:" + chaid + ":.:.:.:"
    seq_header = "sequence:" + model_file_name + ":.:.:.:.:.:.:.:"
    
    # Crea el archivo de alineamiento con la estructura requerida por modeller
    alg_filename = pdb_code + ".alg"
    with open(alg_filename, "w") as handle:
        handle.write("\n>P1;%s\n%s\n%s*\n>P1;%s\n%s\n%s*\n" % (crys_file_name, struc_header, algn1_struc, 
                                                               model_file_name, seq_header, algn2_seq))

    ##########################
    # Ejecuta find_gaps para obtener los posibles gaps de la secuencia
    gaps = find_gaps(algn1_struc, r = num_res_window)
    num_gaps = gaps["num_gaps"]
    # Comprueba si hay gaps
    # if num_gaps != 0:
     #   print("Se omite hacer el modelado de " + pdb_code + ", la proteína ya está completa o tiene más residuos que la secuencia")
     #   print("Longitud de secuencia: " + str(len(algn2_seq)))
     #   print("Longitud de la estructura: " + str(len(algn1_struc)))
     #   continue

    gap_i = gaps["gap_window"]
    # Se define el string que necesita Modeller para definir al primer gap
    s = "self.residue_range('" + str(gap_i[0][0]) + "', '" + str(gap_i[0][1]) + "')"

    # A pesar de que sólo haya un gap, se ejecuta el ciclo, 
    # si hay más de uno, se extiende el String para incluir los demás
    for i in range(1, num_gaps):
        s = s + ", " + "self.residue_range('" + str(gap_i[i][0]) + "', '" + str(gap_i[i][1]) + "')" #nótese la coma
    ##########################
    ''' SE EJECUTA MODELLER'''
    ##########################
    env = environ()
    env.io.atom_files_directory = ['.', struct_dir]
    # Se modifica la clase MyModel de Modeller para fijar los residuos ya existentes en el cristal
    # Estos residuos no se modelarán ni sus átomos cambiarán de posición
    # NECESARIO para poder capturar los valores obtenidos por el rango de gaps
    MyModel_code = """
class MyModel(automodel):
    def select_atoms(self):
        return selection(""" + s + """)
""" # Al estar dentro del loop la identación extra es importante
    exec(MyModel_code) # Se lleva a cabo la modificación a MyModel para agregar la región fijada
    # Se intancia el objeto MyModel
    a = MyModel(env, alnfile = alg_filename, # Lee el archivo fasta creado y guardado en el directorio actual
                      knowns = crys_file_name, # Archivo pdb crys, que coincide con el id en el archivo fasta
                      sequence = model_file_name) # Nombre del modelo
    a.starting_model= 1
    a.ending_model  = 1
    a.library_schedule = autosched.slow # Originalmente comentado
    a.max_var_iterations = 2000 #*500
    a.md_level = refine.slow # Nivel del rifinamiento
    a.repeat_optimization = 3 ##Originalmente comentado
    
    # Más info sobre el refinamiento: https://salilab.org/modeller/9.21/manual/node19.html
    a.make()

    ###########################
    # Renombra el archivo pdb a pdb_code + _full.pdb
    # DEBE ser el único archivo pdb en el directorio '.'

    model_file = glob.glob('./' + pdb_code + '*.pdb')[0] # Nombre del archivo pdb
    os.rename(model_file, pdbs_model_dir + pdb_code + tail_model + ".pdb") # Mueve el pdb a la carpeta model_pdbs
    # Elimina los archivos extra generados - Todos empiezan con el codigo del PDB ID
    for f in glob.glob(pdb_code + "*"):
        os.remove(f)

end = time. time()
print(end - start)

In [5]:
print()

## Como resultado se modelaron 402 estructuras.
